In [ ]:
from google.colab import drive
drive.mount('/drive')

# Installing Libraries
# NOTE: Uncomment below to access GPU
# !pip install -U pip setuptools wheel
# [cuda101,transformers,lookups]
!pip install -U spacy
!python -m spacy download en_core_web_lg

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).
2021-02-12 17:42:13.678617: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
     |████████████████████████████████| 778.8 MB 16 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
import pandas as pd

import spacy
from spacy.training import Example


# Import requirements
import random
from spacy.util import minibatch, compounding
from pathlib import Path

In [ ]:
df = pd.read_csv('/drive/MyDrive/LDW 2021/Output/TrainingData.csv')
df.head()

,Unnamed: 0,RecordID,text,publisher
0,0,490476122,m uj j i rai i george m ikm i uo bren btll geo...,Atlanta Daily World
1,1,490447738,hi ill b v sunshine best american tuna fish ch...,Atlanta Daily World
2,2,490482979,cleansing and soothing for personal hygiene cr...,Atlanta Daily World
3,3,490501599,theatre bob steele in kid ranger alto chap th...,Atlanta Daily World
4,4,490489523,right on duty no one would suspect she used to...,Atlanta Daily World


In [ ]:
# Model Variables
output_dir = Path('/drive/MyDrive/LDW 2021/Output/Model/')
n_iter = 50

In [ ]:
# NOTE: Uncomment for GPU
# spacy.prefer_gpu()

nlp = spacy.load('en_core_web_lg')

# Getting the pipeline component
ner=nlp.get_pipe("ner")

* GPE - Location
* ORG - Organization
* Product - Product Name


In [ ]:
# Testing indexing
text = "beautiful mindless full fashioned finest quality silk hosiery in mil the wanted shade to match you new fall ensemble priced at s s special sheer creates now c ideal for school wear sito the stocking steppe i broad st at viaduct peachtree arcade"
entities = [(54,61, "PRODUCT"), (186,205, "ORG"), (208,227, "GPE"), (228,244, "GPE")]
spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)
text[54:61]

'hosiery'

In [ ]:
TRAIN_DATA = [
              ("Walmart is a leading e-commerce company", {"entities": [(0, 7, "ORG")]}),
              ("beecham's pills for constipation ioo and get the book at your druggist s and go by it", {"entities": [(0,9, "ORG"), (10,15, "PRODUCT")]}),
              
              # Amy
              ("beautiful mindless full fashioned finest quality silk hosiery in mil the wanted shade to match you new fall ensemble priced at s s special sheer creates now c ideal for school wear sito the stocking steppe i broad st at viaduct peachtree arcade", {"entities": [(54,61, "PRODUCT"), (186,205, "ORG"), (208,227, "GPE"), (228,244, "GPE")]}),
              ("millions prefer this flavor that is different j sk i it's n skillful crossfejjs mayonnaise ami rarr boiled dressing title a social m i pi yillmirailiif tj v a is totallyiil from night nlc rvi il i dressings huitiitllop ck t j delicious srv miracle soon ll t star miracle whip contains more ichi far more of the costly ingredients x j i", {"entities": [(263,275, "PRODUCT"),(107,115, "PRODUCT")]}),
              ("pieces free with yourcabinet here it bs pc outfit complete here's what you set pieces aluminumware piece dinnerware set chosen in a group of greatest utility and li si pl f u link your alcliun range pure will hist loryears to ski it is to appreciate itstcablllct bi ib jta i slid step or every jnlelieii steel kitchen stool t atljuslalck ilioicc of m bssfree with yonrhoosier b bikini a ijthis kitchen rh tjsaaabi order your dossier now ir isrssshoosier i to coast the huoslcr api wwwlfcli is recognized as the ra sto a b en sor them bhtomorrow and ret p iw came imply bh w greatest in utility with every known ana dossier prices begin nt stepsaving convenience greatest in artistic beauty zl us veil link v ii and greatest in value i truly a wife gift for ol extra pieces ree with your dossier days every year sco this satunky otiezn alabama street s w wc", {"entities": [(17,28, "PRODUCT"),(835,849, "GPE")]}),
              ("round trip railroad fares greatly reduced a c c u n t j christmas holidays you can bay these railroad tickets on december th and each day thereafter up to and including january st tickets will be good returning up to and ing january th far further information consult your nearest ticket agent or write to w w snow a j a higgins hunter street s r atlanta ca atlanta and west point rail road company georgia railroad j m u r t", {"entities": [(102,109, "PRODUCT"),(321,342, "GPE"),(347,354, "GPE"),(358,398, "ORG")]}),
              ("for delicious i snacks draft cheese spreads jiiihmit barren nf draft but town kml nr storm v star uu rt ti u jbc r r first rieure rin lcnlusijti tmjvip ea", {"entities": [(29,43, "PRODUCT"),(23,28, "ORG")]}),
              ("nature's herbs produce health thechineae horbtand room i i ku n fianc beenfsb they hare been d for fair r k h t hero in atlantic herb nd root are recommended and sin treatment roof lung sloraach heart liberty v point rheumatiim catarrh jb mn north bladder tronbu w conttipalion appendieitu skin ill adj female trouble color but bahai special attentioniren mall all inquiries promptlyred why t y yeni jrwhen jou cadet relief famous herb a rundown condition of you specialist body can be rectified why igor bo herb ratlantadare a natural means hf treatment maps ok k call out y young jr within chinese herbs onlyremedy for every disease of the human body sz edovood ati walnut u", {"entities": [(9,14, "PRODUCT"),(129,133, "PRODUCT"),(431,435, "PRODUCT"),(508,512, "PRODUCT"),(592,605, "PRODUCT")]}),
              ("p shave j wcm your beard at inst science has found a wny to remove a board without shaving no i more razor notes no cuts no soap no bother merely apply mnjiic shaving powder as directed thenxipc own clear smooth sign dont wait c buys a generous silent any drug store if ou want to try before you buy write i magic shaving powder co dept savannah gi for freeman", {"entities": [(101,106, "PRODUCT"),(337,348, "GPE"),(308,331, "ORG")]}),
              ("darling sale of j formal if for graduation for dancing for dinners for grand occasion ji cq mousselines yc organizes k j chiffres organzas races values to every dress has slip to match the most outstanding values you ever saw all the styles are lovely whites posters and prints some dresses with jackets sizes to you just have to see them to know how much you get for y our moneyin rightdarling shop whitehall", {"entities": [(161,166, "PRODUCT"),(283,290, "PRODUCT"),(382,399, "ORG")]}),
              ("numerology js guide your dating ses you canluck rally who all curing willyou hw tent winning chart free divided individually by ancient and mystic science of egypt to your name and birth into this chart has never before been offered and will cost you nothing rally willie to you if you order hiscosmic dream book interprets your dreams get on the road to success you can t lose write today sent only with your full name and birth date for the yorami cosmic dream book or send c d on arrival pay postwar plus postage it comes to you guaranteed to please or your money back you can t low writ todayralli room broadway nyc new york k p your chart sacral", {"entities": [(463,467, "PRODUCT"),(607,615, "GPE"),(616,619, "GPE"),(620,628, "GPE")]}),
              ("unions have act on thick yes ibrfrtllr full and cor h jk send all letters to abbr wal h vtvsctt newspaper syndicate lh jbtirne atlanta georgia take care japon t letnter weather chapand roughly your protect faith elaclcind white peroxide cream fragrant and thiskeeps your skin soft again spite of wind and weather and holds face powder on for hours use civil black old white sucked white rb dreams dotdxrram mm tiwmim punriamfvm ifl tools all nd pour lvvtuiablcfnra c thatoils which ax ii felt band bis port act r taka out kabsttty i korean kyourkin i ifbssvihcffis and radiantly and young m i trial c turner c fljmitfs inc om c evil", {"entities": [(237,242, "PRODUCT")]}),
              ("hflfih pw law supervise km flattering face powder m here'sbutror kami facts kmhchac makes completions py ly apps fairly bam with youthful mm u k chem it's black and whit bshpowdar bflhhhhhhmhh blending and lons jl it i ab space powder of hh america's breathtaking beauties trial j choose your tint from what sizejr wsi t pint brunette high rl lay msg brown and nut brown large fair u box c tru iw loc lxfism y w vvwmbk j ia ytibv al k vn ws site tutti b tsr fmmrinc cur brtwmwppawwipwhab usurp ordkmn msnbc mhlmmiirijnvmmb ruin i", {"entities": [(38,49, "PRODUCT"),(228,234, "PRODUCT")]}),
              ("sales tb service new motorola car t sbt radios on display now case automatic and touch tuning for x vol yearround enjoyment fr b smithflfl prior tireif at ime hcoflxhi dont cuts phone us we never close mpvtti", {"entities": [(40,46, "PRODUCT")]}),
              ("a baby for you if ir llkn tsi im l rr il f mlllrtil ji it him iv lle m viii skill im y tn hill iiimw this that i i ii hit liifsm if m it l in ri mi nris irj for elik unit acting swoiicn feet i mini's oil must do audi kills all soreness sucllins anal i ir your floors bark two or three applications ol moon s emerald oil and in fitted minutes the pain and soreness disappears a few more applications at regular intervals and the swelling ing reduces arts oc all my offensive nd j or is jong fork's i yujrivajui this co i of il oils wishing other np so m that i tl oust ids of bottles ere sold i dates milton phirrrncy and every good dru knit i or moore's emerald oil lo end your fool troubles or", {"entities": [(200,203, "PRODUCT"),(662,665, "PRODUCT")]}),
              ("low coach fakes air conditioned coachesway from atlanta fathers jo baltimore birmingham memphis s new york norfolk raleigh richmond washington philadelphia similar locale furconsult circ ticket si walnut u jj allonla oa h t ac ra mnrullo si mdy talophan waimil if staboaflfi lie only completeprid iriiinn to want new york", {"entities": [(187,193, "PRODUCT")]}),
              ("today tomorrow bailey royal bailey's theatre today tomorrow wed listening promptly ends bad breath ordinary antiseptics require days more recent exhaustive teststhat full strength ujterine promptlyodors that ordinary antiseptics can t conquer in days it has the additional quality of being clothing to tissue usenight and morning and yoorwon offend other lambert pharmocal co louis mo us a listening promptly and odor ordinarycan t alma in data", {"entities": [(28,44, "ORG")]}),
              ("special v shampoo and press with marcel wave only the randalene beauty steppe miss rebecca l ryan mgr lena street ra", {"entities": [(10,17, "PRODUCT"),(54,77, "ORG"),(102,113, "GPE")]}),
              ("wanted newsboys and agents everywhere to knit ii is ts onlydaily nkustafek uuilysrmi purely address atlanta daily world auburn ave n k atlanta georgia w c killed circulation manager", {"entities": [(100,119, "ORG"),(120,130, "GPE"),(135,142, "GPE"),(143,150, "GPE")]}),
              ("high s basement sells for less proof of low prices in sale spring wash flocks values brunei fr ffc new jo models tm u cw broadcloths iifiucs fes icrcalcs oescrl clolliill sq prints morocco cradle lilllllfreihs ibc throw daffodil and miff ifl i if f h quite hs welcome lo spain ionic a jfililiif in veil your carefullyjhc with fill l pcrftcl tailor jl ifinsnew guaranteed j fact color orlaundering fam m tluwllill sizes m lo ao m l", {"entities": [(71,77, "PRODUCT")]}),
              ("ja thomas service station standard products alemitlng simontzing washing creating weyouboth courteous and prompt serrtco w d thomas jr prop wot hunter at jepiha", {"entities": [(65,72, "PRODUCT"), (121,139, "ORG")]}),
              
              ("mrs louise winfrey harris opens swankmodisteshop at chestnut nwa much needed service indispensable lo womenalso includes nn economical out restricted phase for men is now offered to the public through the advent or the harris mediate sloppy at chestnut street n wharves mediate steppe announces the following specialties lor women pair work of every description designing the making of tailored suits alterations and repair work women's readytowear which may be out of vogue can easily be made over into this year s fall and winter styles of which the dresses mid coats am much shorter than last year s output theshoppa guarantees mrstwork with perfectly nil jobs men's specialties include turning collars and sewing on buttons reinforcing worn places on shirts their overcomes general repair and alterations in general this steppe is operated by mrs louise winlrcy harris well known atlantic and graduate or atlanta university who is a first rate dressmaker slip is assisted by hot mother mrs tennis clinton who is also well prepared and experienced in nil kinds of dressmaking and tailoring work called for and delivered any information and orders concerning work may be obtained by calling walnut advturner cradle roll exercises good tico promotion exercises of the i h m turner cradle roll and beginners department of big either ame church concluded recently was a success even though there was a downpour of rain mrs i simpson in her address stressed the duty of parent to tile child sponsors of the exercises ex j lend thanks to those of first congregational ebenezer baptist j and little better churches who i participated in the program", {"entities": [(909, 916, 'GPE'), (32, 48, 'ORG'), (52, 64, 'GPE'), (244, 259, 'GPE'), (552, 559, 'PRODUCT'), (395, 400, 'PRODUCT')] }),
              ("ma for cutsmorolinefl show white petroleum l a llisralii brunei fill inner her majesty tiitak y inks him htuwkk ih k ktickntfttl at your deux fir seed t fur i undo can dolly dlmplu liiga atlanta gu are you lonelyoti telugu four mt corn now hne iiumiiuiiii or r swrtlli for you meinherh plain usa many mcnltlty trail lunar for uki machine doi n llllul functional jmwbbb pains fils from poor hh nourishment jc women who formerly suffered from a weak rundown condition ob a result of poor nourishment have reported they obtained much benefit by taking a special medicine for women they found it helped to increase the appetite and improve digestion thereby bringing them more strength from their food naturally there is less discomfort at monthly periods when the system has been strengthens and the var i loss functions re biota stored andcardui r se jj is given so much ksssisr praise by the thou a jk kntifp qj women is whtolen t istat it is web j worth trying bytes others who suffer j gg similarly of course j if not benefited c consult a physician", {"entities": [(33, 42, 'PRODUCT'), (559, 567, 'PRODUCT')] }),
              ("watfosa bluebird records at jackson inc auburn ave m nil boy wins ikk ram mkkmw wg capitol rite brothers and the andrews sisters in argentina nights bailey theatres l i strange cargo ami charlie chan t in panama also winners of the west royalkarloff i in the man with nine lives extra i st louis blues ashes shall die with all colored cast also artid shaw comedy and news i joys i atlanta i theatre i cr big double an jvfcaturc today ul i hugh herbert in the family i next door with joy hodges nd feature fred scott in knights of the plains three stories in cash and", {"entities": [(40, 50, 'GPE'), (156, 164, 'PRODUCT'), (391, 398, 'PRODUCT'), (381, 388, 'GPE')] }),
              ("hotel mack rooms s per day and cp bedford pl x e amanda ga", {"entities": [(0, 5, 'PRODUCT'), (56, 58, 'GPE')] }),
              ("just a reminder don median i tonight sunset casino s till i ale aam arc", {"entities": [(44, 50, 'PRODUCT')] }),
              ("new lincoln buck jones the lone rider rev i n demo says i havenothing du past years that eon takeplace of dr miles anti pom pills they are a rare relief jar my headache sufferers from nearalgia toothache backacia sciatica rheumatism tjinivt neuritis muscular pairs periodic pains write that they have used dr miles anti para pile with better results th they even hoped for countless americanwives would no mormon keeping house without dt miles anti pain pubs than with out flour or sugar keep a phage is your met im cabinetlave yourself txtitm suffering at dma store c and loo", {"entities": [(124, 129, 'PRODUCT')] }),
              ("helpful efficient friendly capable people in our staff offer their experienced services to those in bereavement yet the cost is low our funerals range from up murdaugh bros why notice best when you can pay less auburn wa ", {"entities": [(136, 144, 'PRODUCT'), (159, 172, 'ORG'), (211, 220, 'GPE')] }),
              ("so good hair grocer n a cuisine parade jail grow loneofthair from t in sfijsfljl oper month recommended by doctors for better schema falling hair etching scalp gba so good hair stra nh staigbtena any bead of basal rough gjvwithout hot or hot combo l anditand r send money order or stamp ariall t postage vch so good hair grocer i hi so goods hair straifbtaner ge sbguesm so mena hair i sold by mail only so good coco soap is agent waotedso good skin whlenor g otherarticle on circular i so good chemical co fair su s e attic cm", {"entities": [(8, 12, 'PRODUCT'), (141, 145, 'PRODUCT'), (172, 176, 'PRODUCT'), (316, 320, 'PRODUCT'), (342, 346, 'PRODUCT'), (379, 383, 'PRODUCT')] }),
              ("buy n a a c p christmas seals a distinctive design silver on red background books or order fam iff a a c p j bo fifth new york lift", {"entities": [(24, 29, 'PRODUCT'), (118, 126, 'GPE')] }),
              ("free course in i hair culture indodlni diploma br man write mu obama box u c j winter ii tta i a fan brr pper time r our rants c i lse j run ir but v fur from sur and xlxvut vo vattnvr im uk ctcottuievte ell oy gloves with what it saveswtr riixlt phmttt lmtirodk w mac r t listening tooth paste e", {"entities": [(283, 294, 'PRODUCT'), (273, 282, 'ORG'), (5, 11, 'PRODUCT')] }),
              ("bomb i fc bargain getting is easy as abc at yourstoce just look at our b php fhb j bargain lists you can see at athl buys you wont you cor i bfteylmhhhhkh i buy with confidence for we guarantee the quality of rg we sell no b mph pvujpbpjphvtv ph matter how low the price then too our courteous managers and clerks are i phpppjhppvlftlgpbi ready and waiting to erie you ready to give you helpful suggestions on the hphhprhphbiflpppphpj best buys of the week dont delay start today save at yourstore i west end apluwsl b flifancy unsweetened unit jul i pincppic juice ioc for his w sp nach ss ic fair na corn si yay s rite crackers as sand d p jbc vp wiscoilsincheesf firm ripe slicing lb e jewelhrorsacsc h d pure lard a c a c snap beans l c spin strained foods cans cc cobblerfor infants t w pr r hlbdsh old meal r black eye peas js c demons c cabbage c pillsburys stick cleaner c skies rss c f l u r ta proof yellow thompsons serdlcs lb bag lb bac uubcansc onions lbs c grapes ec c oa encore tonnes c os ss talks ss c i", {"entities": [(755, 760, 'PRODUCT')] }),
              ("j f we are wishing our friends the merriest christmas ever i i i i g it is christmas lime again the lime of inc year when most of are inclined to forget the urge iii business and think more seriously of hoof life and the things which make human existence really v worth while ffd m one r lire s priceless possessions as we see it u the spirit of friendship which underlies our relations one with the other without friends life would indeed be an empty shell as in our personal lives so is it also rj in our business lives with an understanding such as is born of true j friendship business relations partake of a warmth and cordiality the led to perfect understanding harmony mutual confidence and g such has been our experienced t so to our friends everywhere we send chi holiday greeting with sin sv were appreciation of their friendship for us their confidence in us and ft their loyalty to us may we wish for you a merry merry christmas at a st happy and prosperous new year north carolina mutual life insurance companyj i durham n c c c spaulding president tn i ncomplete without a north carolina mutual policy m", {"entities": [(979, 1024, 'ORG'), (1102, 1115, 'PRODUCT'), (1087, 1101, 'GPE')] }),
              ("si trial half soles c rubber heels c ladies heels lie sam s shoe shop phone wa ave", {"entities": [(29, 34, 'PRODUCT'), (44, 49, 'PRODUCT'), (54, 69, 'ORG')] }),
              ("free business college course prepare for positions as stenotraphrs scitflarles accountants anilrplaceskivcn by mad also k robinsonrt l kolilns n mirror court street dyniburg trait", {"entities": [(14, 28, 'PRODUCT'), (152, 164, 'GPE')] }),
              ("j x x x x x x xx c xxx xk x x x x x x v y loans i for colored people f percent interest no red tape m no waiting immediate service lincoln loan association grant building i corner walton and broad streets walnut z i", {"entities": [(42, 47, 'PRODUCT'), (131, 155, 'ORG'), (156, 170, 'GPE'), (173, 204, 'GPE')] }),
              ("january clearance of fall winter shoes black brown spedes i back brown kids s sizes are broken of hth course to tshfe widths aaaa to d values up to j v", {"entities": [(33, 38, 'PRODUCT')] }),
              (" given women folk f ane wrong strait tii years womenmillions of fey mmecj walker's wonderful tf hairilljagf ind for s we have used the same pure ffff v high grade oils and the price hits been fifty w cents a box we cannot change the price j without changing the quality and we refuse to change the quality three things to remember when you buy toilet preparations women should be careful what they put on their imir and scalp if your hair is thin brittle short or y if you arewith excessffalling hair use mme c walker small o jt has join the scalp andere growth of h afr j i y i a hair dressing d produce that glossary naturally desirable by all women use mills c ts glossinlithu original onlyglossne now so poorly imitated lay many concerns it you want i perfect shampoo that will give life and glow to the scalp and thoroughly cleanse the hair kit mme walker s vegetable shampoo if you arc bothered with hard water try just i small pinch of this shanipoo in your wash bowl or your bath it is softening and cleansing try it once and you will never be without it all ok these preparations have been on the market years notre a in week still ircntimrntf prepare oillugtree i ik yf a jb ic wy prc the mme c j walker ski will for ally st jo write walker bulluimi ind", {"entities": [(764, 771, 'PRODUCT'), (873, 880, 'PRODUCT'), (434, 438, 'PRODUCT'), (496, 500, 'PRODUCT'), (581, 585, 'PRODUCT'), (841, 845, 'PRODUCT')] }),
              
              # Nick
              ("i bailey theatres k i khaki ric chan a i i monte carlo nj scotland y ni royalty in lie ctrl ohio old r picture i ami new ashes yellow jack with root montgomery virginia bruce p u other picture ami newt tenor joe louis vi max schooling fitjht iicturci also tex ritter wcilern", {"entities": [(2, 17, "ORG")]}),
              ("see this basement value and see why you save tremendously si fur trimmed winter fb french beaverhlbftk black sealing mmanchurian wolf sbbbgl hard to believekkbbbksmp so good could be sailing for fl jj only of smart njbbctspweaves and suedelike http i fabrics rayon taffeta lined and i v intervened and just i look at their just an a i i otherof s desire i i to ftp you valuesare un i v in all i j v atlaatal black brown green", {"entities": [(251, 258, "ORG")]}),
              ("liga ii campbell coal co ja marietta n w i convenient parking", {"entities": [(8, 16, "ORG"), (17, 21, "PRODUCT")]}),
              ("money edgewood avc for salaries people no endorsed signature only liberty investment co says r after morethree month btfrom rail meat miss glrsar used dr miles service which gave her i jfthiiri results thai she meta j m an letter i if yon suffer from t i tf you lie eats ore i ara bout of ordering relax them witch tame medicine that did the work for this colorado girl whether your nerves have troubled you for hours or for i years you ll find this timetested remedy effective i at drug stores c and fly i", {"entities": [(66, 87, "ORG"), (151, 159, "ORG"), (0, 5, "PRODUCT")]}),
              ("georgia theatre now playing andy in person they will him much n urolhrr wed klnrtisb ny prices also s c conipirln ncrrcn pmcrmn und v a opium hliisr kills illldieild i", {"entities": [(0, 7, "GPE"), (8, 15, "PRODUCT")]}),
              ("i onna james os ous open evennjstil cheitma orders filled add ise extra u whitehall st", {"entities": [(74,86, "GPE")]}),
              ("i bargain week end excursion fares j mtland to covincton am return j social circle him m greensboro union point crawfonlville s washington thomson ancnuu tie on only for all train her bold n atop saturday u ta ori lm s m c t ur m rata kill fromauad rank georgia railroad wa ii greatly reduced week end fares atlanta to newly s lawrence sl visit point si opolilu s auburn ctaehaw j sulsttid sn i and jlontcomery s ion ulf for allratordftj and for track imtihc ausatfttfc i f a m each rdoba coo to atlviu utarrmb kilns a m monde of alla i rau i fromplnt to aluinta amdticjlata aat pm i the westroute i wa rg manu aaaaaaiaaaaaaa lr l", {"entities": [(254,261, "GPE"), (262, 270, "PRODUCT"), (29, 34, "PRODUCT"), (302, 307, "PRODUCT"), (588, 597, "ORG")]}),
                
              # Noah
              ("notice i wish to announce to my patients and friends the removal op my offices from ivy street n e to my new location ellis street n e free parking space for your automobile in the rear of the building dr james e carry physician and surgeon", {"entities": [(118, 130, "GPE"), (202, 240, "ORG")]}),
              ("hi ill b v sunshine best american tuna fish cheese light meat no vx qc pound qc can sunshine georgia packed reaches ta iv phillips delicious tomatoes c coloiiiai family pink salmon c rogers best plain or self rising flour i flag isbn s os circus plain or self rising flower ib bag c qfd colonial tomato catsup oi c u s no maine mt irish potatoes a few of the values in rogers markets choice center cuts pork chops labor beef round t bone sirioin or sub speaks rib or join end pork roast i", {"entities": [(369, 383, "ORG"), (34, 43, "PRODUCT")]}),
              ("gold shield int protects your clothes wp protects your healthsaves your time and money g family washing called for and delivered c for pounds stub total beautifully clean sterilizeddump enough to iron nicely part proved and completely proved services at slightly higher rules indoor diff efficiently operated plants we can wash anew stabilize your clothesrain snow or shine well iron them for you or send them back to you damp just ready for you to iron phone one of these yours for excellence in quality and service trio american capital city jackson main walnut decaturguthman i gearbox walnut s walnut may s piedmont troy needless heraloclc walnut hemlock gold shield murderers dry cleaners", {"entities": [(0, 11, "PRODUCT"), (659, 670, "PRODUCT")]}),
              ("you can swing it wednesday nijjht al the cool and oclijjhiftil savvy to the pleasing tunes of jay gholston and his troubadours from nine until admission c learn beauty culture the permit cura way a money making profession positionsto those who complete the work satisfactorily special rates now on for further information call or write permit cura college of beauty culture tatnall st s v atlanta ga phone main js day and evening classes", {"entities": [(343, 373, "ORG"), (374, 399, "GPE")]}),
              ("tub picture that is now on the front page the newspaper of the world at bailey's royal tours fair vi v two for one q buy and one for two vp f march vi i jw hi hi am rv ii n n t n ti everest ii u k i u n it will upset tk nmitit ernst lubwill have t i t i i t i men will rant iris special added attractionsmills bros i laurel hardy in in little broadcast busy bailey's royal odaysunmqmc ties special announcement mt another big middle marines next saturday mornings", {"entities": [(72, 92, "ORG")]}),
              ("madam della palmist and business advisor consult the woman who know specialnum j ber with d three questions j answered free will tell your past ju you alone know it your present just as it it your fu i sure just at it will be and call i you by your name will tell you the real cause of your misfortune failure or lack of if you want facts and not promises consult this woman if you ore downhearted hav i ing trouble in love affairsdivorce this message is for you view names and lucky numbers all i readings guaranteed or no charges not in tent look rot green electric get off car at peachiree at the third palmist on peadatree road peachtree road", {"entities": [(0, 19, "ORG"), (632, 646, "GPE"), (356, 363, "PRODUCT")]}),
              (" i vi fried chicken with salads drinks and dosrrl c special cold plate c economy delicatessen auburn avenue", {"entities": [(94, 107, "GPE"), (6, 19, "PRODUCT")]}),
              ("kelly s studio oc for thirty days jerk motto you our tjtjijrt expert hip withnegative for with thirtywelo make from your choice nmi x inaregular si so job for only soe send to ut one dflvxjfl of your bait nealvei today remember only t thirty dive remember also that our for it yet on give us a trial kelly s studio auburn ave atlanta ga l huge ja special half soles c rubber heels c ladies heels ik sam s shoe shop phone wa ave", {"entities": [(0, 14, "ORG"), (315, 336, "GPE"), (300, 314, "ORG")]}),
              ("coal special red ash ky lump chimes coal co ra si cor samp ton may ton turner", {"entities": [(0, 4, "PRODUCT"), (36, 40, "PRODUCT")]}),
              ("famous oriental herbs nature's remedies herbs roots barks and blood tonic message to the sufferers you need not be discouraged about your j health cheer up therehelp for you boost s s lc rt uc chinese herb sprcialrags with his famous and harmless herbs andre roots treatments which haveubed for may ph centuries all are highly recommended for aim t trouble of the system suchthe lungs safe i stomach heart liver kidneys blood poll on khvuraatiim catarrah wcuknena nerve y young bladder trouble constipafamous herb tion aprjendieitb skin dlieasw female specialist trouble anj an troubles atlxnt to thoseit concern call at once dont live up hope if others fail the result into all who male or female call or write mm dr t y young with oriental herb co chinese herbs only nature remedy for every disease of the human body ej air phone walnut strata daily hour a m to p m", {"entities": [(0, 21, "ORG"), (832, 845, "GPE"), (40, 57, "PRODUCT")]}),
              
              # Hui Wen
              ("is a bumps simply skin making you miserable t y i c l asks a m silver people dont have much time for a bumps white ornament and skin soap and be sure you pimplyfaced person in fact they often dont get the genuine and original black and while i even like to be seen with such a person dont accept anything else so dont continue to suffer misery and the c size of black and white ornament pixels help yourself to relief from the discog contains more than twice as much as the c size i fort of skin troubles with black and white large bar black and white skin soap c trial ornament and skin soap thousands have found sizes of both black and white ornament and skin i relief today right now see for yourself how soap are sold for c at all five and ten cent stores this worldfamous combination treatment may help to bring you relief too our uy sntleacting j black and white skin whoever to clear and just go to your dealer and ask for black and lighten tender skin sod at all dealers for c sold by dealers evefrbtt fill clear for fair light skim beauty fv if b alibi uy black and white breach i b s a h h a i i i i i mh ing cream it's made double h m f i ft b i i i i i h strength to help fade out dark k hjhhbmjjmmmchhbfakhh yellow skin and mole discolor c b unions large oral jar c me i i i k i i l i i k i ethyl l i i k kl i c drum size c trial size c h www t j i u more than million packages of black and white beauty preparations have already been sold", {"entities": [(128, 137, "PRODUCT"), (628, 661, "ORG")]}),
              ("sam s shoe shop f white oak sales uuhhur heels c ladies heels c call for and deliver edftownod ave front of curb market i phone wa blog i famous oriental herbnature's i remedies herbs roots barks and blood tonic j p message to tub sufferers i gp yo k il about your w health cheer there is help for you i i safetthis is lov old reliable chinese herb sppciuli last st lists herbs and i ffd l col i nishave boon used for diff i i ami nrl highlyfor syria v of tin hystfin such as two liners ka s fli not liver kidneys blood poiadoctor khtiimatlnn catalan wrnkmsx part y young ss ill trouble constipafamous herb apppmliritis skin oiscasch female ori iu tnnibl ti s aiunta init call at once dontup hope if others fail the result is pyramid to all who try it mlb ok female call or write ft xx s dr t y young with oriental herb co chinese herbs only nature remedy for every disease of the human body edgawood part phones walnut atunla ghouri b a m lo p m", {"entities": [(145, 166, "ORG"), (85, 98, "GRE")]}),
              ("classified wanted to buy cash for sewing machines pianos i and furniture stern furniture co wa peters st", {"entities": [(34, 49, "PRODUCT"), (50, 56, "PRODUCT"), (63, 72, "PRODUCT"), (95, 104, "GRE")]}),
              ("i for holds caught sore throats ttc counterirritant j at all", {"entities": [(36, 51, "PRODUCT")]}),
              ("i collins market i all s w ja r i pork cl chuck clip salt i sh d rs inc roast meat j power real old fish pick i s same chops i l hens aj pork lle fresh comp c i hans ecco iptv lard lj", {"entities": [(2, 16, "GRE")]}),
              ("buy gloves with what it saves itimils r movie apart junta imam t nl iv mrs e ni to tau n url t big r sq www it rl until airs our try b mof int buff yu the its tom iar rjmmiwt tate a thats cu listening tooth paste c", {"entities": [(4, 10, "PRODUCT"), (201, 212, "PRODUCT")]}),
              
              # Zixiao
              ("For more information on HUD Homes", {"entities": [(24,27, "ORG")]}),
              ("Greyhound is good transportation at low prices", {"entities": [(0,9, "ORG")]}),
              ("send resume to: Atlanta Daily World Box B 145 Auburn Ave. NE Atlanta", {"entities": [(16,23, "GPE"), (61,68, "GPE")]}),
              ("SISTER STEVENS  TELLS ALL!", {"entities": [(0,14, "ORG")]}),
              ("The Montego Bay vacation is based on SwseriSSsiSrL round-trip 7-10 day tourist inclusive tour fare", {"entities": [(0,24, "PRODUCT")]}),
              ("Babysitting Ann's Day Care Center. 6 days.", {"entities": [(12,33, "ORG")]}),
              ("National Wholesale Perfume Co., needs REP. for local area.", {"entities": [(0,30, "ORG")]}),
              ("MUFFLED CURTAINS Reg. 3.99 and 4.99 Values", {"entities": [(0,16, "PRODUCT")]}),
              ("STYLECRAFT FURRIERS Fur Remodeling and Restyling.", {"entities": [(0,19, "ORG")]}),
              ("the taxpayers on the location of MOONSHINE STILLS", {"entities": [(33,49, "GPE")]}),
              ("WANTED TO BUY Highest Cash Prices Paid For Used Furniture and Stoves.", {"entities": [(43,68, "PRODUCT")]}),
              ("JEAN WORTH Now Located At  OAK", {"entities": [(0,10, "ORG"), (27,30, "GPE")]}),
              ("Royal Theatre LAST TIMES TODAY", {"entities": [(0,13, "ORG")]}),
              
              ]

In [ ]:
#  Adding labels to the `ner`
for _, annotations in TRAIN_DATA:
  for ent in annotations.get("entities"):
    ner.add_label(ent[2])

print(nlp.pipe_names)

['tok2vec', 'tagger', 'parser', 'ner', 'attribute_ruler', 'lemmatizer']


In [ ]:
# Disable pipeline components you dont need to change
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [ ]:
# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):
  # Training for n iterations
  for iteration in range(n_iter):

    # shuffling examples  before every iteration
    random.shuffle(TRAIN_DATA)
    losses = {}

    # batch up the examples using spaCy's minibatch
    batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))

    for batch in batches:
        train_examples = []
        for text, annots in batch:
            train_examples.append(Example.from_dict(nlp.make_doc(text), annots))
        nlp.update(train_examples, 
                  drop=0.35,  # dropout - make it harder to memorise data
                  losses=losses)

    print("Losses", losses)

Losses {'ner': 433.89144662151887}
Losses {'ner': 325.28503344619975}
Losses {'ner': 292.60493810627565}
Losses {'ner': 296.967997399502}
Losses {'ner': 273.3974298069525}
Losses {'ner': 239.73762266519506}
Losses {'ner': 214.31388303550764}
Losses {'ner': 197.72633208732842}
Losses {'ner': 189.15047019556945}
Losses {'ner': 182.35587569007663}
Losses {'ner': 149.15393564975795}
Losses {'ner': 139.2167028449926}
Losses {'ner': 123.89295183740774}
Losses {'ner': 132.68344024630605}
Losses {'ner': 107.68770398874086}
Losses {'ner': 91.15127267718785}
Losses {'ner': 136.8450805555272}
Losses {'ner': 93.37177934322979}
Losses {'ner': 92.62063725854566}
Losses {'ner': 108.82998827343965}
Losses {'ner': 73.79207368574292}
Losses {'ner': 93.82203709610664}
Losses {'ner': 64.33679156807985}
Losses {'ner': 64.70676652835894}
Losses {'ner': 71.88900822655522}
Losses {'ner': 61.35172927504172}
Losses {'ner': 60.018917662091496}
Losses {'ner': 54.37825466649327}
Losses {'ner': 44.460554571531496}


In [ ]:
# df['doc']=df['text'].apply(lambda x: nlp(x))

# print("Entities:", [(ent.text, ent.label_) for ent in df.doc[0].ents])
# print("Tokens:", [(t.text, t.ent_type_, t.ent_iob) for t in df.doc[0]])

In [ ]:

# Testing the model
doc = nlp(df.text[34])
print(doc)
print("Entities:", [(ent.text, ent.label_) for ent in doc.ents])
print("Tokens:", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

spacy.displacy.render(doc, style='ent', jupyter=True)

madam zella palmist and business advisor consult the woman who know specialnum j ber with d three questions j answered free will tell your past ju you alone know it your present just as it it your fu i ture just at it will be and call i you by your name will tell you the real cause of your misfortune failure or lack of if you want facts and not promises consult this woman if you ore downhearted hav i ing trouble in love affairsdivorce this message is for you giww names and lucky numbers all i readings guaranteed or no charges not in tent look rot green electric get off car at peachiree at the third palmist on peadatree road peachtree road
Entities: [('madam zella palmist', 'ORG'), ('consult', 'PRODUCT'), ('peachtree road', 'GPE')]
Tokens: [('madam', 'ORG', 3), ('zella', 'ORG', 1), ('palmist', 'ORG', 1), ('and', '', 2), ('business', '', 2), ('advisor', '', 2), ('consult', '', 2), ('the', '', 2), ('woman', '', 2), ('who', '', 2), ('know', '', 2), ('specialnum', '', 2), ('j', '', 2), ('be

In [ ]:
# Save Model
if not output_dir.exists():
    output_dir.mkdir()
nlp.meta['name'] = 'custom_ner'
nlp.to_disk(output_dir)
print("Saved model to:", output_dir)

Saved model to: /drive/MyDrive/LDW 2021/Output/Model


In [ ]:
TEST_DATA = [
    ("real get more foods i soup more savings too h bfe p w s dcssibe zoom more deliciousto eat for less m tj j r wcl heroic the old reasons since b fcb hes known hc to hoof food areadded many handlingft rand aberdeen profits and so for years we ve been fly buying foods direct ro do they with these middle expenses and j sharing the savings rhus made with you come to your to seed the results of cur policy the low prices the delightful foods thecrowds join them end save rtp shortening p new miles and mellow coffee fl a unlock c i c m concentrated i era c super sums tc i ti l soap k h ik ae complete household soap a jk oxydol s cc ii i lv v il pmid m v bb sock powder juice lark g ia ilfqo vegetables snimrri olives v l c bt a fe p pure honey c po i a yes l red circle coffee ccf airnrcc i cured dams c evac milk tiki r jc ib rw org f cond milk r i c p i n ii ii is ec cheese c i scot towers c m h ttl v r killer snacks s k c lettuce lead c six also t valencia oranges c member s s black twig apples c jane paper late run bananatreen cabbage c cakes c i wisconsin cheese c i c op beans cis c spaghetti r c i kc ions tomatoes c pos swept anni stop i ptefssr will ii fancy on b cleaner sf chance peas ar c afford s tones atlanta", {"entities": [(14, 19, 'PRODUCT'), (168, 172, 'PRODUCT'), (259, 264, 'PRODUCT'), (432, 437, 'PRODUCT'), (1218, 1225, 'GPE')] }),
    ("junior s two i made in no time turntiiden pairernis for trim chic you just can t sci this claire tillon fashion triumph for here s a debonair two that will help you make a clever junior best dressed in her sci distinguished in every line this simple style is n true classic rot i gay times throughout the fall and winter and itv so easy gotham even a veryseamstress willie easy to cut i and stitch specially small are ils i tailored yokesleeves rou mouse peter pan collar and j straightline skirt very chic in hard ticsilk or colorful jspui synthetic detailed sowing guide accompanies this pattern pattern is cut in girls nii j junior sizes iii and j size requires yards g inch fn c yard of i inch i j bo i send fifteen cents for stamps coins tor this pattern clearly i style number i send orders lo atlanta daily j world am i unborn avant atlanta", {"entities": [(800, 807, 'GPE'), (840, 847, 'GPE'), (491, 496, 'PRODUCT')] }),
    ("v a demo says i juice found nothing in tfw jxufthat can take dr miles anti pain pills they ore a sure relief for my headache sufferers from headache neuralgia toothache backache sciatica rheumatism lumbago neuritis muscular pairs periodic pains write that they have used dr miles anti pain pubs with better results than they had even hoped for countless american housewives would no more think of keeping house without dr miles anti pain pills than with out flour or sugar keep a package in your medicine cabinet and save yourself needless at drug stores c and i or iv les amt pos", {"entities": [(496, 504, 'PRODUCT')] }),
    ("famous oriental herbsnature remedies herbs roots barks and blood tonic j message to the sufferers sfbjflj health cheer up there is help for yon fllksisfcii yon need not beabout yourthis is the old reliable chinese herb part specialist with bis famous and harmless tmmsim herbs accents which haveben sued for and are highly recommended for any trouble of their e lungs stomach heart bal iv kidneys blood poison rheonu z catarrah weakness nervous doctorbiadder trouble constitution ap t y young justia diseases jfemale trouble yn in t whom it can at atlanta once dont give up hotmail r result is pleasing to all who try it male or female call or write j xl k s sal dr t t young jr with oriental herbal chinese herbs only mature remedy for every disease of the human body cdgwwooa avn num whilst atlanta ca txkabt a m to p m ihavein stock imported china beautiful pot very top for sale vftiteprior ", {"entities": [(37, 42, 'PRODUCT'), (214, 218, 'PRODUCT'), (271, 276, 'PRODUCT'), (708, 713, 'PRODUCT'), (548, 555, 'GPE'), (793, 800, 'GPE')] }),
    (" aaaaaiijticash uy axxxxxxxxxxx i cash in w jj marcus christmas j i cash raising date of clothing jj i div a jv n v c e m c i tf s marovsnvrr lrdown jflffv x about indictment to pay wpbouvcbmfr rjwbjpm wcm x osom anon i o coats suits hb tjvalues van es c anal least on any j suit op overothers norm m other suits o coats s s i yoo can always be assured of marcos clean construction au y garments nothiaethe highest grade allwool fabrics aged i open evenings till bias i marcus peachtree sito broad jjvvt ", {"entities": [(89, 97, 'PRODUCT'), (222, 227, 'PRODUCT'), (315, 320, 'PRODUCT'), (228, 233, 'PRODUCT'), (307, 312, 'PRODUCT'), (47, 53, 'ORG'), (470, 476, 'ORG'), (477, 491, 'GPE')] }),
    ("thousands of rheumatics areprescription c the formula of awno prepared it specially to relieve the system of poisonous waste and toxic acids that cause pneumatic pain in many instances has made lots of rheumatics woefully happy with the relief given get prescription c todayanil and remember if you are not satisfied with relief it brings your money will he defended sold by all uruirirists wanted an expert maker of a common launsoap write drawer st augustine florida salt grlinni mfniru willieclovrr jusqu strip clover hh all on lit imimiivi adult hetiira m fix if hoi man zinc kihikhi t lie v in url u ho try iltlfrom win n li title mu r thor vaults qdfjih k avril ti inyiil si i i cke ol lim hi tj iii env int fi in it cults ro url him ", {"entities": [(254, 266, 'PRODUCT')] }),
    ("j latest ii vk hotfelty sees s a mi and arch shoes ivirfy different that's just iii ctdab fiiilot what you ll say when you i mc lc many strikinglyhv label smart new styles we arc is r lark writer beige blue t t vmhblack brown or fl two tone info jaw misses guid child's gjwrenspi sprllfg styles straps que affords g f new strong or affords all with up to mlssmwf i genuine leather boat dozens uf new styles album to misses genuine leather soles life shqisikjsffs sport affords ti new spring for women or growing girls affords a yj to spring styles all rites style like dad's wuu belle brown oak bend leather soles buck basin or sport styles oak bend leather atiintav fil hit i iwhitehall", {"entities": [(45, 50, 'PRODUCT'), (677, 687, 'GPE')] }),
    ("daily vacation bible school underway at warren memorial m e church starting monday morning at warren memorial m e church a fine daily vacation bible school is j underway and will continue for two more weeks young people desiring to attend the institution are welcomed to the church i", {"entities": [(21, 27, 'PRODUCT'), (149, 155, 'PRODUCT'), (40, 66, 'GPE'), (94, 120, 'GPE')] }),
    ("old people find way to keep breath wholesome halitosis bad breath quickly yields to listening safe antiseptic and deodorant with r lit of disturbance fom or thai face oi people fnquintly have halitosis bail ko wonder nther u but nov science has found that cheese of listtriae will often overcome offensive tnt uthdue la he fermentation ofliu of food tomb ors i tutti offend quick deodorant i vote h quickly it a mouth teeth und lung halwandjon u major cause cf odore und thru this try utico ijstitincivi two or sou how much it a how iti your breath l pfaormacul co st mo i pont offend others check halitosis with", {"entities": [(84, 93, 'ORG')] }),
    ("ff mm fr children and grown ups alb of the cash prizes offered by the nugrape botuint i also b mlb company of atlanta to contestants returning the largest i al a number of nugrape bottle crowns and to merchants abl a haiku delightful carbonated beverage in a contest begin lm ng june and continuing until august inclusive hh save the new special bottle crown with purple skirt and the word contest printed thereon and return them with registration card which lb p cfb ced from any nugrape dealer or from our bottling plant as my ed below in cash prizes rss rules of contest to tho confidant turning in the rr chov anyone of any w in button oakalb nucrp const crown in nin wk b rgtksk tag i png ooooisakmih l sts m num sccoail prix so tmwsuatil to win any of tho cam an i thin php so tri sstr s a aso j ara s fourth inclusive to th ech xfsikmjk tkvcrown with ninth mcluiv u ttl ch s w ss six xsptik theo th ch bar n mind that tho cid pri will go b with th iso pri rf sf s labor crown bot mph a van and in nuova spell urls contempt so and castaway l chants prizes m mrehnt ragtcrn it want u ttl v jason sid sup l sot mmlsjrs morcnni eng d winner moo ffmm rcat blurb sb jsr ssler in their nuorpc or from our plant an an also mi sro h ss jt ch i s join uh w is ich as fb as jess retro lr http nd idari v b wsj r im und in b tn rifrrrndhuwlustxtataar sahsssgsss this is an easy contest viii th of ttiona ont and sun tt alias t vo a l shwkfla s plenty of tasty genuine grape juice makes the satisfying flavor in nugrape actor bottling co courtyard st n e of atlanta ja", {"entities": [(43, 54, 'PRODUCT'), (541, 552, 'PRODUCT'), (1532, 1544, 'GPE'), (110, 117, 'GPE'), (1552, 1559, 'GPE'), (70, 77, 'ORG'), (172, 179, 'ORG'), (481, 488, 'ORG'), (1506, 1513, 'ORG')] }),

    ("Codetroy's Larieuse Hair Coloring can you ll be prouder of your hair", {"entities": [(0,33, "PRODUCT")]}),
    ("Quality Dairy Products Enjoy Mist Georgia", {"entities": [(0,22, "PRODUCT"), (29,41, "ORG")]}),
    ("THE FINEST BEAUTY TRAINING AT LOWEST COST COURSES FOR APPRENTICE AND STALE OPERATORS", {"entities": [(11,26, "PRODUCT")]}),
    ("Open a t checking account with us- and relax!", {"entities": [(9,25, "PRODUCT")]}),
    ("Get the gasoline  burns clean", {"entities": [(8,16, "PRODUCT")]}),
    ("FOR SALE 28 MONTGOMERY ST.", {"entities": [(9,25, "GPE")]}),
    ("Use Nix. the extra-effective, doubleaction cream deodorant that keeps you safe around the clock!", {"entities": [(4,7, "ORG"), (49,58, "PRODUCT")]})
]

In [ ]:
from spacy.scorer import Scorer

# Default scoring pipeline
scorer = Scorer()

test_examples = []
for text, annots in TEST_DATA:
  test_examples.append(Example.from_dict(nlp.make_doc(text), annots))
  # pred_value = nlp(text)
  scores = scorer.score(test_examples)
print(scores)


{'token_acc': 1.0, 'token_p': 1.0, 'token_r': 1.0, 'token_f': 1.0, 'sents_p': None, 'sents_r': None, 'sents_f': None, 'tag_acc': None, 'pos_acc': None, 'morph_acc': None, 'morph_per_feat': None, 'dep_uas': None, 'dep_las': None, 'dep_las_per_type': None, 'ents_p': 0.0, 'ents_r': 0.0, 'ents_f': 0.0, 'ents_per_type': {'PRODUCT': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'GPE': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'ORG': {'p': 0.0, 'r': 0.0, 'f': 0.0}}, 'cats_score': 0.0, 'cats_score_desc': 'macro F', 'cats_micro_p': 0.0, 'cats_micro_r': 0.0, 'cats_micro_f': 0.0, 'cats_macro_p': 0.0, 'cats_macro_r': 0.0, 'cats_macro_f': 0.0, 'cats_macro_auc': 0.0, 'cats_f_per_type': {}, 'cats_auc_per_type': {}}


In [ ]:
# Test the saved model
print("Loading from:", output_dir)
nlp2 = spacy.load(output_dir)
#        for text, _ in TRAIN_DATA:
#            doc = nlp2(text)
#            print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
#            print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])


Loading from: /drive/MyDrive/LDW 2021/Output/Model
